<a href="https://colab.research.google.com/github/chrishamblin7/dimensionality_reduction_and_superposition/blob/main/latent_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Latent Data in 'Superposition'

This notebook includes code adopted to perform experiments from the 'Toy models of Superposition paper'. Given simulated data with known 'real' features, we will create embeddings of those features in a lower dimensional space such that read out of the original features can be performed with minimal error.

This notebook is designed to run in Google Colab's Python 3.7 environment.

In [1]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from torch import nn
from torch.nn import functional as F

from typing import Optional

from dataclasses import dataclass, replace
import numpy as np
import einops

from tqdm.notebook import trange

import time
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from google.colab import files

In [3]:
@dataclass
class Config:
  n_features: int
  n_hidden: int

  # We optimize n_instances models in a single training loop
  # to let us sweep over sparsity or importance curves 
  # efficiently.

  # We could potentially use torch.vmap instead.
  n_instances: int
 
class Model(nn.Module):
  def __init__(self, 
               config, 
               feature_probability: Optional[torch.Tensor] = None,
               importance: Optional[torch.Tensor] = None,               
               device='cuda'):
    super().__init__()
    self.config = config
    self.W = nn.Parameter(torch.empty((config.n_instances, config.n_features, config.n_hidden), device=device))
    nn.init.xavier_normal_(self.W)
    self.b_final = nn.Parameter(torch.zeros((config.n_instances, config.n_features), device=device))

    if feature_probability is None:
      feature_probability = torch.ones(())
    self.feature_probability = feature_probability.to(device)
    if importance is None:
      importance = torch.ones(())
    self.importance = importance.to(device)

  def forward(self, features):
    # features: [..., instance, n_features]
    # W: [instance, n_features, n_hidden]
    hidden = torch.einsum("...if,ifh->...ih", features, self.W)
    out = torch.einsum("...ih,ifh->...if", hidden, self.W)
    out = out + self.b_final
    out = F.relu(out)
    return out

  def generate_batch(self, n_batch):
    feat = torch.rand((n_batch, self.config.n_instances, self.config.n_features), device=self.W.device)
    batch = torch.where(
        torch.rand((n_batch, self.config.n_instances, self.config.n_features), device=self.W.device) <= self.feature_probability,
        feat,
        torch.zeros((), device=self.W.device),
    )
    return batch

In [4]:
def linear_lr(step, steps):
  return (1 - (step / steps))

def constant_lr(*_):
  return 1.0

def cosine_decay_lr(step, steps):
  return np.cos(0.5 * np.pi * step / (steps - 1))

def optimize(model, 
             render=False, 
             n_batch=1024,
             steps=10_000,
             print_freq=100,
             lr=1e-3,
             lr_scale=constant_lr,
             hooks=[]):
  cfg = model.config

  opt = torch.optim.AdamW(list(model.parameters()), lr=lr)

  start = time.time()
  with trange(steps) as t:
    for step in t:
      step_lr = lr * lr_scale(step, steps)
      for group in opt.param_groups:
        group['lr'] = step_lr
      opt.zero_grad(set_to_none=True)
      batch = model.generate_batch(n_batch)
      out = model(batch)
      error = (model.importance*(batch.abs() - out)**2)
      loss = einops.reduce(error, 'b i f -> i', 'mean').sum()
      loss.backward()
      opt.step()
    
      if hooks:
        hook_data = dict(model=model,
                         step=step, 
                         opt=opt,
                         error=error,
                         loss=loss,
                         lr=step_lr)
        for h in hooks:
          h(hook_data)
      if step % print_freq == 0 or (step + 1 == steps):
        t.set_postfix(
            loss=loss.item() / cfg.n_instances,
            lr=step_lr,
        )

In [5]:
if torch.cuda.is_available():
  DEVICE = 'cuda'
else:
  DEVICE = 'cpu'

## Generate data

We will optimize the relu encoder/ decoder model, then pull the weights of
the encoder to get the projection of N features into M (M<N) dimensions
at different sparsities

In [6]:
n_features = 60 #number of original orthogonal features
n_instances = 6 #number of sparsities (linearly spaced between 1 and .05)
n_hiddens = [50,40,30,20,10,5] #number of compressed features, well test a range of values here


importance_sweep = True

In [7]:
(20 ** -torch.linspace(0, 1, n_instances))[:, None]

tensor([[1.0000],
        [0.5493],
        [0.3017],
        [0.1657],
        [0.0910],
        [0.0500]])

In [8]:
if importance_sweep:
  importance = (0.9**torch.arange(n_features))[None, :]  
  imp_name = 'withimportance'
else:
  importance = torch.ones(n_features)[None,:]
  imp_name = ''



for n_hidden in n_hiddens:
  print(n_hidden)

  config = Config(
      n_features = n_features,
      n_hidden = n_hidden,
      n_instances = n_instances,
  )

  model = Model(
      config=config,
      device=DEVICE,
      # Exponential feature importance curve from 1 to 1/100
      importance = importance,
      # Sweep feature frequency across the instances from 1 (fully dense) to 1/20
      feature_probability = (20 ** -torch.linspace(0, 1, n_instances))[:, None]
  )

  optimize(model)

  WA = model.W.detach()

  outname = str(n_features)+'to'+str(n_hidden)+'_'+imp_name+'.npy'

  np.save(outname,WA.detach().cpu().numpy())

  files.download(outname) #download data to googledrive


50


  0%|          | 0/10000 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

40


...

<IPython.core.display.Javascript object>

5


  0%|          | 0/10000 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
optimize(model)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
WA = model.W.detach()
WA.shape

torch.Size([10, 30, 10])

In [ ]:
import numpy as np

np.save('30t10_dense.npy',WA[0].detach().cpu().numpy())
np.save('30t10_sparse.npy',WA[0].detach().cpu().numpy())